In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Info'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp)
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//table[@class="moreinfotable"]/tr')[1:]
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'Info': input_.loc[a, 'Info']}])

                for tr in list_tr:
                    df_temp.loc[0, tr.xpath('./td[1]')[0].text.lower().strip()] = tr.xpath('./td[2]')[0].text.strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Info': input_.loc[a, 'Info']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Info'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Info'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./info-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Info'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./info_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：310

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=937003
[尝试次数：1] - [剩余数量：300] - [当前时间：18:25:29]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=929301
[尝试次数：2] - [剩余数量：299] - [当前时间：18:25:30]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=1193592
[尝试次数：3] - [剩余数量：298] - [当前时间：18:25:30]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=942614
[尝试次数：4] - [剩余数量：297] - [当前时间：18:25:31]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=939835
[尝试次数：1] - [剩余数量：296] - [当前时间：18:25:31]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=934289
[尝试次数：2] - [剩余数量：295] - [当前时间：18:25:31]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=1185107
[尝试次数：6] - [剩余数量：294] - [当前时间：18:25:31]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=935238
[尝试次数：4] - [剩余数量：293] - [当前时间：18:25:32]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=1188452
[尝试次数：7] - [剩余数量：292] - [当前时间：18:25:32]

[ok] - https://www.rockau

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=931783
[尝试次数：14] - [剩余数量：223] - [当前时间：18:25:52]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=1192581
[尝试次数：1] - [剩余数量：222] - [当前时间：18:25:52]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=1193390
[尝试次数：1] - [剩余数量：221] - [当前时间：18:25:53]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=932457
[尝试次数：2] - [剩余数量：220] - [当前时间：18:25:53]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=944075
[尝试次数：1] - [剩余数量：219] - [当前时间：18:25:53]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=1186983
[尝试次数：3] - [剩余数量：218] - [当前时间：18:25:54]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=1185548
[尝试次数：1] - [剩余数量：217] - [当前时间：18:25:54]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=944625
[尝试次数：1] - [剩余数量：216] - [当前时间：18:25:54]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=930819
[尝试次数：15] - [剩余数量：215] - [当前时间：18:25:55]

[ok] - https://www.rockauto.com

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=2064762
[尝试次数：4] - [剩余数量：144] - [当前时间：18:26:17]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=11553617
[尝试次数：1] - [剩余数量：143] - [当前时间：18:26:17]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=2066986
[尝试次数：2] - [剩余数量：142] - [当前时间：18:26:17]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=942190
[尝试次数：1] - [剩余数量：141] - [当前时间：18:26:18]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=944189
[尝试次数：1] - [剩余数量：140] - [当前时间：18:26:18]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=3970632
[尝试次数：2] - [剩余数量：139] - [当前时间：18:26:19]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=3613752
[尝试次数：1] - [剩余数量：138] - [当前时间：18:26:19]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=932848
[尝试次数：1] - [剩余数量：137] - [当前时间：18:26:19]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=937837
[尝试次数：5] - [剩余数量：136] - [当前时间：18:26:19]

[ok] - https://www.rockauto.com

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=940003
[尝试次数：5] - [剩余数量：67] - [当前时间：18:26:43]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=931295
[尝试次数：3] - [剩余数量：66] - [当前时间：18:26:44]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=941803
[尝试次数：13] - [剩余数量：65] - [当前时间：18:26:44]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=14867481
[尝试次数：3] - [剩余数量：64] - [当前时间：18:26:45]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=936519
[尝试次数：15] - [剩余数量：63] - [当前时间：18:26:45]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=14867473
[尝试次数：6] - [剩余数量：62] - [当前时间：18:26:45]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=14867513
[尝试次数：2] - [剩余数量：61] - [当前时间：18:26:46]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=14867497
[尝试次数：4] - [剩余数量：60] - [当前时间：18:26:46]

[ok] - https://www.rockauto.com/en/moreinfo.php?pt=4748&pk=14867529
[尝试次数：1] - [剩余数量：59] - [当前时间：18:26:46]

[ok] - https://www.rockauto.com/en